In [9]:
!pip install tf2onnx
!pip install tensorflow_ranking

In [10]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Reshape, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
import tensorflow as tf
import tensorflow_ranking as tfr
import tf2onnx
from keras.utils.np_utils import to_categorical

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
input_path = 'drive/MyDrive/CURIS/train_networks/gtsrb.pickle'
output_path = 'drive/MyDrive/CURIS/networks/'

In [13]:
# Read and process dataset
import pickle
with open(input_path, 'rb') as handle:
    gtsrb = pickle.load(handle)

x_train, y_train, x_valid, y_valid, x_test, y_test = gtsrb.values()
x_train = x_train/255
x_valid = x_valid/255
x_test = x_test/255

y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)
y_test = to_categorical(y_test)

In [14]:
def eval_and_save(model):
  score = model.evaluate(x_test, y_test, verbose=0)
  print("Test loss:", score[0])
  print("Test accuracy:", score[1])
  model.summary()
  model.save(output_path + model.name + '.h5')
  model_proto, _ = tf2onnx.convert.from_keras(model, output_path=output_path + model.name + '.onnx')

10x2 fully connected

In [15]:
model = Sequential(name='gtsrb-10x2')
model.add(Flatten(name='input'))
model.add(Dense(10, name='dense_1', activation='relu'))
model.add(Dense(10, name='dense_2', activation='relu'))
model.add(Dense(10, name='logit'))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_valid, y_valid))

Epoch 1/20
138/138 [==============================] - 4s 10ms/step - loss: 1.8236 - accuracy: 0.3897 - val_loss: 1.4556 - val_accuracy: 0.5676
Epoch 2/20
138/138 [==============================] - 1s 7ms/step - loss: 1.1314 - accuracy: 0.6684 - val_loss: 1.1024 - val_accuracy: 0.6686
Epoch 3/20
138/138 [==============================] - 1s 9ms/step - loss: 0.8089 - accuracy: 0.7715 - val_loss: 0.9242 - val_accuracy: 0.7106
Epoch 4/20
138/138 [==============================] - 1s 7ms/step - loss: 0.6552 - accuracy: 0.8116 - val_loss: 0.7762 - val_accuracy: 0.7415
Epoch 5/20
138/138 [==============================] - 1s 7ms/step - loss: 0.5597 - accuracy: 0.8402 - val_loss: 0.6896 - val_accuracy: 0.7894
Epoch 6/20
138/138 [==============================] - 1s 8ms/step - loss: 0.5018 - accuracy: 0.8591 - val_loss: 0.5887 - val_accuracy: 0.8585
Epoch 7/20
138/138 [==============================] - 1s 8ms/step - loss: 0.4420 - accuracy: 0.8802 - val_loss: 0.6007 - val_accuracy: 0.8585
Epoch

In [16]:
eval_and_save(model)

Test loss: 0.49579766392707825
Test accuracy: 0.872376561164856
Model: "gtsrb-10x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 30)                92190     
                                                                 
 dense_2 (Dense)             (None, 30)                930       
                                                                 
 logit (Dense)               (None, 10)                310       
                                                                 
Total params: 93,430
Trainable params: 93,430
Non-trainable params: 0
_________________________________________________________________


30x2 fully connected

In [17]:
model = Sequential(name='gtsrb-30x2')
model.add(Flatten(name='input'))
model.add(Dense(30, name='dense_1', activation='relu'))
model.add(Dense(30, name='dense_2', activation='relu'))
model.add(Dense(10, name='logit'))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_valid, y_valid))

Epoch 1/20
138/138 [==============================] - 2s 6ms/step - loss: 1.7233 - accuracy: 0.4636 - val_loss: 1.3570 - val_accuracy: 0.5729
Epoch 2/20
138/138 [==============================] - 1s 4ms/step - loss: 0.9919 - accuracy: 0.7393 - val_loss: 0.9688 - val_accuracy: 0.7150
Epoch 3/20
138/138 [==============================] - 1s 4ms/step - loss: 0.7223 - accuracy: 0.8120 - val_loss: 0.7545 - val_accuracy: 0.8068
Epoch 4/20
138/138 [==============================] - 1s 4ms/step - loss: 0.5762 - accuracy: 0.8465 - val_loss: 0.6539 - val_accuracy: 0.8048
Epoch 5/20
138/138 [==============================] - 1s 4ms/step - loss: 0.4784 - accuracy: 0.8738 - val_loss: 0.6172 - val_accuracy: 0.8198
Epoch 6/20
138/138 [==============================] - 1s 4ms/step - loss: 0.4181 - accuracy: 0.8888 - val_loss: 0.5253 - val_accuracy: 0.8570
Epoch 7/20
138/138 [==============================] - 1s 4ms/step - loss: 0.3678 - accuracy: 0.9051 - val_loss: 0.5002 - val_accuracy: 0.8556
Epoch 

In [18]:
eval_and_save(model)

Test loss: 0.3951394855976105
Test accuracy: 0.9037036895751953
Model: "gtsrb-30x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 30)                92190     
                                                                 
 dense_2 (Dense)             (None, 30)                930       
                                                                 
 logit (Dense)               (None, 10)                310       
                                                                 
Total params: 93,430
Trainable params: 93,430
Non-trainable params: 0
_________________________________________________________________


100x2 fully connected

In [ ]:
model = Sequential(name='gtsrb-100x2')
model.add(Flatten(name='input'))
model.add(Dense(100, name='dense_1', activation='relu'))
model.add(Dense(100, name='dense_2', activation='relu'))
model.add(Dense(10, name='logit'))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_valid, y_valid))

In [ ]:
eval_and_save(model)

Test loss: 0.3781207799911499
Test accuracy: 0.9141975045204163
Model: "gtsrb-100x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 100)               307300    
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 logit (Dense)               (None, 10)                1010      
                                                                 
Total params: 318,410
Trainable params: 318,410
Non-trainable params: 0
_________________________________________________________________


simple cnn model

In [ ]:
x_test.shape

(6480, 32, 32, 3)

In [ ]:
model = Sequential(name='gtsrb-cnn-3x3')
model.add(Conv2D(4, (3, 3), name='conv_1', input_shape=(32, 32, 3)))
model.add(Conv2D(4, (3, 3), name='conv_2'))
model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(10, name='logit'))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_valid, y_valid))

In [ ]:
eval_and_save(model)

Test loss: 0.5874148607254028
Test accuracy: 0.917438268661499
Model: "gtsrb-cnn-3x3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 30, 30, 4)         112       
                                                                 
 conv_2 (Conv2D)             (None, 28, 28, 4)         148       
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 20)                62740     
                                                                 
 logit (Dense)               (None, 10)                210       
                                                                 
Total params: 63,210
Trainable params: 63,210
Non-trainable params: 0
____________________________________________________

cnn with max pooling and dropout

In [ ]:
model = Sequential(name='gtsrb-sota')
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dense(10, name='logit'))
model.add(Activation('softmax', name='output'))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_valid, y_valid))

Epoch 1/20
138/138 [==============================] - 74s 516ms/step - loss: 2.0482 - accuracy: 0.4092 - val_loss: 1.9173 - val_accuracy: 0.5401
Epoch 2/20
138/138 [==============================] - 65s 472ms/step - loss: 1.8160 - accuracy: 0.6475 - val_loss: 1.7178 - val_accuracy: 0.7411
Epoch 3/20
138/138 [==============================] - 70s 507ms/step - loss: 1.6879 - accuracy: 0.7754 - val_loss: 1.6865 - val_accuracy: 0.7720
Epoch 4/20
138/138 [==============================] - 65s 469ms/step - loss: 1.6669 - accuracy: 0.7948 - val_loss: 1.6714 - val_accuracy: 0.7903
Epoch 5/20
138/138 [==============================] - 68s 497ms/step - loss: 1.6522 - accuracy: 0.8091 - val_loss: 1.5955 - val_accuracy: 0.8667
Epoch 6/20
138/138 [==============================] - 68s 496ms/step - loss: 1.5700 - accuracy: 0.8926 - val_loss: 1.5599 - val_accuracy: 0.9029
Epoch 7/20
138/138 [==============================] - 68s 496ms/step - loss: 1.5552 - accuracy: 0.9062 - val_loss: 1.5543 - val_ac

In [ ]:
eval_and_save(model)

Test loss: 1.4854918718338013
Test accuracy: 0.9760802388191223
Model: "gtsrb-sota"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0  